# Function Health Clinical Tests vs. Omics Data Correlation Analysis

This notebook analyzes correlations between matched Function Health and Arivale clinical chemistry tests with Arivale omics data (proteomics and metabolomics).

## Objectives:
1. Load matched clinical test mappings
2. Load and align omics data (proteomics and metabolomics)
3. Calculate correlations between clinical tests and omics features
4. Identify statistically significant associations
5. Visualize and report findings

## 1. Setup and Imports

In [10]:
#%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.stats.multitest import multipletests
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# Set plot style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

## 2. Load Data

In [11]:
# Load chemistry data
chemistry_df = pd.read_csv('/procedure/data/local_data/ARIVALE_SNAPSHOTS/chemistries.tsv', 
                          sep='\t', comment='#')
print(f"\nLoaded chemistry data: {chemistry_df.shape}")

# Load proteomics data
proteomics_df = pd.read_csv('/procedure/data/local_data/ARIVALE_SNAPSHOTS/proteomics_corrected.tsv', 
                           sep='\t', comment='#')
print(f"Loaded proteomics data: {proteomics_df.shape}")

# Load metabolomics data  
metabolomics_df = pd.read_csv('/procedure/data/local_data/ARIVALE_SNAPSHOTS/metabolomics_corrected.tsv',
                             sep='\t', comment='#')
print(f"Loaded metabolomics data: {metabolomics_df.shape}")


Loaded chemistry data: (11167, 140)
Loaded proteomics data: (6018, 1216)
Loaded metabolomics data: (3225, 1304)


## 3. Data Preprocessing

In [12]:
# Get list of matched Arivale test names
matched_arivale_tests = matched_tests['Display Name'].dropna().unique()
print(f"\nNumber of matched Arivale tests: {len(matched_arivale_tests)}")

# Get chemistry columns (excluding metadata columns)
chem_cols = [col for col in chemistry_df.columns if col not in ['public_client_id', 'vendor', 'vendor_observation_id', 
                                                                   'observation_id', 'reflexive', 'fasting', 
                                                                   'days_in_program', 'days_since_first_call', 
                                                                   'days_since_first_draw', 'month', 'weekday', 'season',
                                                                   'sample_id']]
print(f"\nTotal chemistry columns: {len(chem_cols)}")
print("First 10 chemistry columns:", chem_cols[:10])


Number of matched Arivale tests: 128

Total chemistry columns: 128
First 10 chemistry columns: ['A/G RATIO', 'ADIPONECTIN, SERUM', 'ALAT (SGPT)', 'ALBUMIN', 'ALKALINE PHOSPHATASE', 'ANTIOXID CAP, TOTAL', 'ARACHIDONIC ACID', 'ARSENIC, BLOOD', 'ASAT (SGOT)', 'BASOPHILS']


In [13]:
# Load the mapping results
mapping_df = pd.read_csv('/home/ubuntu/biomapper/data/function_health/function_health_arivale_merged_outer.csv')
print(f"Loaded {len(mapping_df)} test mappings")
print("\nMatched tests:")
matched_tests = mapping_df[mapping_df['Match_Type'] != 'No Match']
print(matched_tests[['Test Name', 'Display Name', 'Match_Type']].head(20))

Loaded 331 test mappings

Matched tests:
                     Test Name            Display Name  \
0            Apolipoprotein A1                 Protein   
1   Arachidonic Acid/EPA Ratio        Arachidonic acid   
2                      Arsenic                 Arsenic   
3       White Blood Cell Count  White Blood Cell Count   
4                         Zinc                    Zinc   
5              Total Bilirubin         Total Bilirubin   
6            Total Cholesterol       Total cholesterol   
7                Total Protein           Total Protein   
8                Triglycerides           Triglycerides   
9                    Uric Acid               Uric Acid   
10                   Vitamin D               Vitamin D   
11                      Sodium                  Sodium   
12        Red Blood Cell Count    Red Blood Cell Count   
13                    Selenium                Selenium   
14              Platelet Count          Platelet Count   
15                   Potassium 

In [14]:
# Let's check the mapping in more detail
print("\nDetailed mapping check:")
print("Sample matched Display Names from mapping file:")
print(matched_arivale_tests[:10])

print("\n\nMatching process:")
found_matches = []
for i, display_name in enumerate(matched_arivale_tests[:20]):  # Check first 20
    found = False
    # Direct match
    if display_name in chem_cols:
        found_matches.append((display_name, display_name, 'Direct'))
        found = True
    # Uppercase match
    elif display_name.upper() in chem_cols:
        found_matches.append((display_name, display_name.upper(), 'Upper'))
        found = True
    # Check for Name column match (from mapping file)
    else:
        # Get the Name from the mapping file for this Display Name
        name_match = mapping_df[mapping_df['Display Name'] == display_name]['Name'].values
        if len(name_match) > 0:
            arivale_name = name_match[0]
            if arivale_name in chem_cols:
                found_matches.append((display_name, arivale_name, 'Via Name'))
                found = True
    
    if not found and i < 10:
        print(f"  No match for: {display_name}")

print(f"\nFound {len(found_matches)} matches")
for orig, matched, match_type in found_matches[:10]:
    print(f"  '{orig}' -> '{matched}' ({match_type})")


Detailed mapping check:
Sample matched Display Names from mapping file:
['Protein' 'Arachidonic acid' 'Arsenic' 'White Blood Cell Count' 'Zinc'
 'Total Bilirubin' 'Total cholesterol' 'Total Protein' 'Triglycerides'
 'Uric Acid']


Matching process:

Found 20 matches
  'Protein' -> 'PROTEIN' (Upper)
  'Arachidonic acid' -> 'ARACHIDONIC ACID' (Upper)
  'Arsenic' -> 'ARSENIC, BLOOD' (Via Name)
  'White Blood Cell Count' -> 'WHITE CELL COUNT' (Via Name)
  'Zinc' -> 'ZINC' (Upper)
  'Total Bilirubin' -> 'BILIRUBIN, TOTAL' (Via Name)
  'Total cholesterol' -> 'CHOLESTEROL, TOTAL' (Via Name)
  'Total Protein' -> 'PROTEIN, TOTAL SERUM' (Via Name)
  'Triglycerides' -> 'TRIGLYCERIDES' (Upper)
  'Uric Acid' -> 'URIC ACID' (Upper)


In [ ]:
# Create merged dataset with chemistry and omics data
# Check if sample_id exists in chemistry_df
if 'sample_id' not in chemistry_df.columns:
    # If no sample_id, we'll need to create a unique identifier
    chemistry_df['sample_id'] = chemistry_df['public_client_id'] + '_' + chemistry_df.index.astype(str)
    print("Created sample_id column in chemistry data")

# Check if sample_id exists in proteomics and metabolomics
if 'sample_id' not in proteomics_df.columns:
    print("Warning: sample_id not found in proteomics data")
if 'sample_id' not in metabolomics_df.columns:
    print("Warning: sample_id not found in metabolomics data")

# First, merge chemistry with proteomics
merged_df = pd.merge(
    chemistry_df[['public_client_id', 'sample_id'] + chemistry_test_cols],
    proteomics_df,
    on=['public_client_id', 'sample_id'],
    how='inner',
    suffixes=('_chem', '_prot')
)
print(f"\nAfter merging chemistry + proteomics: {merged_df.shape}")

# Then merge with metabolomics
merged_df = pd.merge(
    merged_df,
    metabolomics_df,
    on=['public_client_id', 'sample_id'],
    how='inner',
    suffixes=('', '_metab')
)
print(f"After merging with metabolomics: {merged_df.shape}")
print(f"\nTotal samples with all data types: {len(merged_df)}")

In [15]:
# Create improved mapping between Display Names and chemistry column names
display_to_chem = {}
for display_name in matched_arivale_tests:
    # Direct match
    if display_name in chem_cols:
        display_to_chem[display_name] = display_name
    # Uppercase match
    elif display_name.upper() in chem_cols:
        display_to_chem[display_name] = display_name.upper()
    # Check for Name column match (from mapping file)
    else:
        # Get the Name from the mapping file for this Display Name
        name_match = mapping_df[mapping_df['Display Name'] == display_name]['Name'].values
        if len(name_match) > 0:
            arivale_name = name_match[0]
            if arivale_name in chem_cols:
                display_to_chem[display_name] = arivale_name

# Get the actual chemistry columns that we can analyze
chemistry_test_cols = list(set(display_to_chem.values()))  # Use set to remove duplicates
print(f"\nFound {len(chemistry_test_cols)} unique matched test columns in chemistry data")
print("Matched columns:", chemistry_test_cols[:10], "...")

# Create reverse mapping for reporting
chem_to_display = {v: k for k, v in display_to_chem.items()}


Found 128 unique matched test columns in chemistry data
Matched columns: ['VLDL TRIGLYCERIDES', 'VLDL PARTICLE NUMBER', 'GGT', 'TOTAL NEUTROPHILS', 'LP_PLA2', 'WHITE CELL COUNT', 'LEAD, BLOOD', 'PAI-1 ANTIGEN, QNT', 'VITAMIN D3, 25-OH', 'LDL SMALL'] ...


In [ ]:
# Create merged dataset with chemistry and omics data
# First, merge chemistry with proteomics
merged_df = pd.merge(
    chemistry_df[['public_client_id', 'sample_id'] + chemistry_test_cols],
    proteomics_df,
    on=['public_client_id', 'sample_id'],
    how='inner',
    suffixes=('_chem', '_prot')
)
print(f"\nAfter merging chemistry + proteomics: {merged_df.shape}")

# Then merge with metabolomics
merged_df = pd.merge(
    merged_df,
    metabolomics_df,
    on=['public_client_id', 'sample_id'],
    how='inner',
    suffixes=('', '_metab')
)
print(f"After merging with metabolomics: {merged_df.shape}")
print(f"\nTotal samples with all data types: {len(merged_df)}")

## 4. Correlation Analysis

In [ ]:
# Identify omics feature columns
# Proteomics columns typically start with chip IDs like 'CAM_', 'CRE_', etc.
proteomics_cols = [col for col in merged_df.columns if any(col.startswith(prefix) for prefix in 
                   ['CAM_', 'CRE_', 'CVD2_', 'CVD3_', 'DEV_', 'INF_', 'IRE_', 'MET_', 'NEU1_', 
                    'NEX_', 'ODA_', 'ONC2_', 'ONC3_'])]

# Metabolomics columns are numeric (metabolite IDs)
metabolomics_cols = [col for col in merged_df.columns if col.isdigit() or col.startswith('100')]

print(f"\nIdentified {len(proteomics_cols)} proteomics features")
print(f"Identified {len(metabolomics_cols)} metabolomics features")
print(f"\nExample proteomics features: {proteomics_cols[:5]}")
print(f"Example metabolomics features: {metabolomics_cols[:5]}")

In [ ]:
# Function to calculate correlations for a clinical test
def calculate_test_correlations(test_name, omics_features, data_df, omics_type='proteomics'):
    """
    Calculate correlations between a clinical test and all omics features
    """
    results = []
    
    # Get test values
    if test_name not in data_df.columns:
        return pd.DataFrame()
    
    test_values = data_df[test_name].dropna()
    
    for feature in omics_features:
        if feature not in data_df.columns:
            continue
            
        # Get feature values
        feature_values = data_df[feature]
        
        # Find common non-null indices
        common_idx = test_values.index.intersection(feature_values.dropna().index)
        
        if len(common_idx) < 10:  # Require at least 10 samples
            continue
            
        # Calculate correlation
        x = test_values.loc[common_idx]
        y = feature_values.loc[common_idx]
        
        # Pearson correlation
        r, p_value = stats.pearsonr(x, y)
        
        # Spearman correlation (non-parametric)
        rho, p_spearman = stats.spearmanr(x, y)
        
        results.append({
            'clinical_test': test_name,
            'omics_feature': feature,
            'omics_type': omics_type,
            'n_samples': len(common_idx),
            'pearson_r': r,
            'pearson_p': p_value,
            'spearman_rho': rho,
            'spearman_p': p_spearman
        })
    
    return pd.DataFrame(results)

In [ ]:
# Calculate correlations for all matched tests
all_correlations = []

for test in chemistry_test_cols[:5]:  # Start with first 5 tests for demo
    print(f"\nCalculating correlations for {test}...")
    
    # Proteomics correlations
    prot_corr = calculate_test_correlations(test, proteomics_cols[:100], merged_df, 'proteomics')
    all_correlations.append(prot_corr)
    
    # Metabolomics correlations
    metab_corr = calculate_test_correlations(test, metabolomics_cols[:100], merged_df, 'metabolomics')
    all_correlations.append(metab_corr)
    
    print(f"  - Found {len(prot_corr)} proteomics correlations")
    print(f"  - Found {len(metab_corr)} metabolomics correlations")

# Combine all results
correlation_results = pd.concat(all_correlations, ignore_index=True)
print(f"\nTotal correlations calculated: {len(correlation_results)}")

## 5. Multiple Testing Correction

In [ ]:
# Apply FDR correction for multiple testing
if len(correlation_results) > 0:
    # For Pearson correlations
    _, pvals_corrected, _, _ = multipletests(correlation_results['pearson_p'], 
                                             alpha=0.05, method='fdr_bh')
    correlation_results['pearson_q'] = pvals_corrected
    
    # For Spearman correlations
    _, pvals_corrected_sp, _, _ = multipletests(correlation_results['spearman_p'], 
                                                alpha=0.05, method='fdr_bh')
    correlation_results['spearman_q'] = pvals_corrected_sp
    
    # Filter for significant correlations
    significant_corr = correlation_results[
        (correlation_results['pearson_q'] < 0.05) & 
        (np.abs(correlation_results['pearson_r']) > 0.3)
    ].sort_values('pearson_r', key=abs, ascending=False)
    
    print(f"\nSignificant correlations (FDR < 0.05, |r| > 0.3): {len(significant_corr)}")
    print("\nTop 10 significant correlations:")
    print(significant_corr[['clinical_test', 'omics_feature', 'omics_type', 
                           'pearson_r', 'pearson_q', 'n_samples']].head(10))

## 6. Visualization

In [ ]:
# Create volcano plot for one test
if len(correlation_results) > 0 and chemistry_test_cols:
    test_to_plot = chemistry_test_cols[0]
    test_data = correlation_results[correlation_results['clinical_test'] == test_to_plot]
    
    if len(test_data) > 0:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        
        # Proteomics volcano plot
        prot_data = test_data[test_data['omics_type'] == 'proteomics']
        if len(prot_data) > 0:
            ax1.scatter(prot_data['pearson_r'], -np.log10(prot_data['pearson_p']), 
                       alpha=0.6, s=30)
            ax1.axhline(y=-np.log10(0.05), color='r', linestyle='--', alpha=0.5)
            ax1.axvline(x=0.3, color='g', linestyle='--', alpha=0.5)
            ax1.axvline(x=-0.3, color='g', linestyle='--', alpha=0.5)
            ax1.set_xlabel('Pearson Correlation (r)')
            ax1.set_ylabel('-log10(p-value)')
            ax1.set_title(f'{test_to_plot} vs Proteomics')
        
        # Metabolomics volcano plot
        metab_data = test_data[test_data['omics_type'] == 'metabolomics']
        if len(metab_data) > 0:
            ax2.scatter(metab_data['pearson_r'], -np.log10(metab_data['pearson_p']), 
                       alpha=0.6, s=30, color='orange')
            ax2.axhline(y=-np.log10(0.05), color='r', linestyle='--', alpha=0.5)
            ax2.axvline(x=0.3, color='g', linestyle='--', alpha=0.5)
            ax2.axvline(x=-0.3, color='g', linestyle='--', alpha=0.5)
            ax2.set_xlabel('Pearson Correlation (r)')
            ax2.set_ylabel('-log10(p-value)')
            ax2.set_title(f'{test_to_plot} vs Metabolomics')
        
        plt.tight_layout()
        plt.show()

In [ ]:
# Heatmap of top correlations
if len(significant_corr) > 0:
    # Pivot data for heatmap
    top_features = significant_corr.groupby('clinical_test').head(5)
    
    heatmap_data = top_features.pivot_table(
        values='pearson_r',
        index='omics_feature',
        columns='clinical_test'
    )
    
    if not heatmap_data.empty:
        plt.figure(figsize=(12, 8))
        sns.heatmap(heatmap_data, cmap='RdBu_r', center=0, 
                   vmin=-1, vmax=1, cbar_kws={'label': 'Pearson r'})
        plt.title('Top Omics-Clinical Test Correlations')
        plt.tight_layout()
        plt.show()

## 7. Example: Detailed Analysis for Glucose

In [ ]:
# Look for glucose test
glucose_matches = [test for test in chemistry_test_cols if 'glucose' in test.lower()]
print("Glucose-related tests:", glucose_matches)

if glucose_matches:
    glucose_test = glucose_matches[0]
    
    # Calculate full correlations for glucose
    print(f"\nCalculating all correlations for {glucose_test}...")
    glucose_prot = calculate_test_correlations(glucose_test, proteomics_cols, merged_df, 'proteomics')
    glucose_metab = calculate_test_correlations(glucose_test, metabolomics_cols, merged_df, 'metabolomics')
    
    glucose_all = pd.concat([glucose_prot, glucose_metab])
    
    # Apply FDR correction
    _, pvals_corrected, _, _ = multipletests(glucose_all['pearson_p'], alpha=0.05, method='fdr_bh')
    glucose_all['pearson_q'] = pvals_corrected
    
    # Get top correlations
    glucose_sig = glucose_all[
        (glucose_all['pearson_q'] < 0.05) & 
        (np.abs(glucose_all['pearson_r']) > 0.3)
    ].sort_values('pearson_r', key=abs, ascending=False)
    
    print(f"\nTop glucose correlations:")
    print(glucose_sig[['omics_feature', 'omics_type', 'pearson_r', 'pearson_q']].head(20))

## 8. Summary Report Generation

In [ ]:
# Generate summary report
if len(correlation_results) > 0:
    print("\n=== CORRELATION ANALYSIS SUMMARY ===")
    print(f"\nTotal clinical tests analyzed: {correlation_results['clinical_test'].nunique()}")
    print(f"Total omics features tested: {correlation_results['omics_feature'].nunique()}")
    print(f"Total correlations calculated: {len(correlation_results)}")
    
    # Summary by omics type
    print("\nCorrelations by omics type:")
    print(correlation_results.groupby('omics_type').size())
    
    # Significant findings
    sig_results = correlation_results[correlation_results['pearson_q'] < 0.05]
    print(f"\nSignificant correlations (FDR < 0.05): {len(sig_results)}")
    
    # Strong correlations
    strong_corr = sig_results[np.abs(sig_results['pearson_r']) > 0.5]
    print(f"Strong correlations (|r| > 0.5, FDR < 0.05): {len(strong_corr)}")
    
    # Summary by clinical test
    print("\nSignificant correlations per clinical test:")
    test_summary = sig_results.groupby('clinical_test').agg({
        'omics_feature': 'count',
        'pearson_r': lambda x: f"{x.abs().max():.3f}"
    }).rename(columns={'omics_feature': 'n_significant', 'pearson_r': 'max_abs_r'})
    print(test_summary)

In [ ]:
# Save results
if len(correlation_results) > 0:
    # Save all correlations
    correlation_results.to_csv('omics_clinical_correlations_all.csv', index=False)
    print("\nSaved all correlations to 'omics_clinical_correlations_all.csv'")
    
    # Save significant correlations
    if len(significant_corr) > 0:
        significant_corr.to_csv('omics_clinical_correlations_significant.csv', index=False)
        print(f"Saved {len(significant_corr)} significant correlations to 'omics_clinical_correlations_significant.csv'")

## 9. Next Steps and Recommendations

Based on this analysis, next steps include:

1. **Pathway Analysis**: For proteins/metabolites showing strong correlations, perform pathway enrichment analysis
2. **Time Series Analysis**: Account for longitudinal nature of data (multiple timepoints per patient)
3. **Machine Learning**: Build predictive models using omics features to predict clinical test values
4. **Clinical Interpretation**: Work with domain experts to interpret biological significance
5. **Expand Analysis**: Run full analysis on all matched tests (not just demo subset)
6. **Network Analysis**: Build correlation networks to identify clusters of related features
7. **Genomics Integration**: If genomic data available, identify genetic variants associated with test variations